# . . . . . . . . . . . . . . . . . . . . . . . . . . . .
# Decision Tree Linear Regression
# . . . . . . . . . . . . . . . . . . . . . . . . . . . .




Afin de créer un arbre de décision, nous allons étudier un CSV sur les revenus d'individus aux USA.
Cela contient les informations sur le status marital, l'âge, le type d'emploi etc.
Les datas sont de 1994. Nous voulons predire si le salaire sera inferieur voire égal à 50k ou supérieur à 50k

Voilà la source : [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Adult)
Le CSV est inclus.

### <span style="color:purple">Importe le fichier income.csv fourni
 </span>

In [12]:
import pandas

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)
income.head(15)

FileNotFoundError: File b'income.csv' does not exist

### <span style="color:purple">Combien d'observations y a t-il ?
 </span>

In [13]:
print(income.shape[0])

NameError: name 'income' is not defined

 <span style="color:green">Attendu : 32561</span>

### ------------------------------------------------------------------------------------------------------------------------------

## Catégories

### <span style="color:purple">Prends le temps d'interpreter les datas. Quels sont les differents facteurs, les	Θ. Sous quelle forme sont elles classifiées.<br> Sex donne "Male" ou "Female". "workclass" contient plusieurs sortes de réponses.   Afin de pouvoir les analyser nous allons les convertirs en valeures numériques en fonction des categories.   Pour cela utilise la class [pandas.Categorical()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Categorical.html)</span>

In [82]:
col = pandas.Categorical(income["workclass"])
income["workclass"] = col.codes
income.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0


### <span style="color:purple">Convertis le reste des catgorical columns dans `income` (education, marital_status, occupation, relationship, race, sex, native_country, and high_income) en categories numerique.</span>


In [83]:
to_convert = ["education","marital_status","occupation","relationship","race","sex","native_country","high_income"]
for convert in to_convert :
    new_col = pandas.Categorical(income[convert])
    income[convert] = new_col.codes
income.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0


### <span style="color:purple">A present divise income en deux `DataFrames` en fonction de `workclass` afin de diviser ceux travaillant dans le secteur privé ou non. </span>
*python hint : Booleans to split a dataframe.*

In [84]:
private_incomes = income[income["workclass"] == 4]
public_incomes  = income[income["workclass"] != 4]

In [85]:
private_incomes.head(2)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0


In [86]:
public_incomes.head(2)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0


### ------------------------------------------------------------------------------------------------------------------------------

###  <span style="color:purple">Calcule une proportion / probabilite</span>
 ##### <span style="color:purple">Quelle est la proportion de `private_income` et de `public_incomes` ?
 </span>

In [87]:
private_incomes_prop = private_incomes.shape[0]/income.shape[0]
public_incomes_prop = public_incomes.shape[0]/income.shape[0]

print("private_incomes_proportion",private_incomes_prop)
print("public_incomes_proportion",public_incomes_prop)

private_incomes_proportion 0.6970301894904948
public_incomes_proportion 0.3029698105095052


 <span style="color:green">private_incomes 0.6970301894904948 <br>
public_incomes 0.3029698105095052</span>

### ------------------------------------------------------------------------------------------------------------------------------

## Entropy

$$-\sum _{ i=1 }^{ c }{ P({ x }_{ i })\log _{ b }{ P({ x }_{ i }) }  } $$

In [88]:
import numpy

def calc_entropy(column):

    # Détermine le nombre d'occurence de chacune des valeurs dans la colonne
    counts = numpy.bincount(column)
    # détermine la probabilité d'occurence de chacune des catégories
    probabilities = counts / len(column)
    
    entropy = 0
    # fais une boucle par probabilités/proportions et additionne les résultats
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    entropy = -entropy
    return entropy

print("high_income", calc_entropy(income["high_income"]))
print("workclass", calc_entropy(income["workclass"] ))

high_income 0.796383955202
workclass 1.64797692751


 <span style="color:green">Attendu : <br> high_income 0.796383955202 <br>
workclass 1.64797692751</span>

### ------------------------------------------------------------------------------------------------------------------------------

## Information Gain

###  <span style="color:purple">Calcule le `Information Gain` de `age` en fonction de l'objectif final `high_income`</span>
 
$$Entropy(T)\quad =\sum _{ i=1 }^{ c }{ P({ x }_{ i })\log _{ b }{ P({ x }_{ i }) }  } \\ \\ IG(T,\quad A)\quad =\quad Entropy(T)-\sum _{ v\epsilon A }^{  }{ \frac { |{ T }_{ v }| }{ |T| }  } .Entropy({ T }_{ v })$$


In [89]:
import math

#trouve le médian de age.
median_age = income["age"].median()

# crée deux subset en fonction du médian
left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

# calcule la proportion de chacuns des splits.
left_prop = left_split.shape[0]/income.shape[0]
right_prop = right_split.shape[0]/income.shape[0]

In [90]:
# Calcule l'entropie de high_income l'objectif final
income_entropy = calc_entropy(income["high_income"])


In [91]:
#calculez l' `age_information_gain`
age_information_gain = income_entropy - (left_prop * calc_entropy(left_split["high_income"])
                                         + right_prop * calc_entropy(right_split["high_income"]))
age_information_gain

0.047028661304691965

 <span style="color:green">Attendu:0.047028661304691965</span>

 ### <span style="color:purple">Créer une fonction `calc_information_gain`</span>

In [92]:
def calc_information_gain(data, split_name, target_name):

    # calcule l'entropy d'origine
    original_entropy = calc_entropy(data[target_name])

    # trouve le médiant de la colonne
    column = data[split_name]
    median = column.median()
    
    # crée deux subset en fonction du médian
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # calcule le subset entropy de chacun des set
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Retourne information gain
    return original_entropy - to_subtract

# Vérifie que la valeur coïncide avec `income`, "age", "high_income"
print(calc_information_gain(income, "age", "high_income"))

0.0470286613047


 <span style="color:green">Attendu: 0.0470286613047</span>

 ### <span style="color:purple">Puis une `liste information_gains` de toutes les colonnes</span>

In [93]:
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
information_gains = []
#faire une boucle
for col in columns:
    information_gain = calc_information_gain(income, col, "high_income")
    information_gains.append(information_gain)

### ------------------------------------------------------------------------------------------------------------------------------

 ### <span style="color:purple">Selectionne le nom de colonne avec la valeur la plus élevée</span>

In [94]:
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
highest_gain

'marital_status'

### ------------------------------------------------------------------------------------------------------------------------------